In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os

os.environ['SPOTIPY_CLIENT_ID']='3e8627e438fa4781abf064e0d3bc7b6f'
os.environ['SPOTIPY_CLIENT_SECRET']='0ac3077f50b54e248fc32c64c8fc9e13'

# initialize spotipy
client_credentials_manager = SpotifyClientCredentials()
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# read in data from playlist_name_cleaned.csv
df = pd.read_csv('playlist_name_cleaned.csv')

# randomly select rows from the dataset
# df = df.sample(n=10, random_state=1)

In [45]:
import fasttext 
from fasttext import train_supervised

# convert df to dictionary
playlist_dict = df.set_index('playlist_name').T.to_dict('list')

for key in list(playlist_dict.keys()):
    tracklist = eval(playlist_dict[key][0])
    playlist_dict[key] = tracklist

# place each key on a new line in a text file
with open('playlist_name_cleaned.txt', 'w') as f:
    for key in playlist_dict.keys():
        f.write(key + ' ')
        for track in playlist_dict[key]:
            f.write('__label__' + track + ' ')
        f.write('\n')

# train fasttext model on the playlist names, using track uri as the label
model = fasttext.train_supervised('playlist_name_cleaned.txt')

# test model, predict the top 5 labels for a playlist name
pred = model.predict("rock and roll")
pred_string = str(pred[0])
# remove '__label__' from the beginning of each label
pred_string = pred_string.replace('__label__', '')
# remove the brackets from the beginning and end of the string
pred_string = "spotify:track:" + pred_string[2:-3]
print(pred_string)
print(spotify.track("spotify:track:" + pred_string)['name'])

/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_14451/4135848979.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = df.set_index('playlist_name').T.to_dict('list')


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [5]:
songs = ['15MTd64KUMG7CF6mOyovsQ', '3kGfazcbVvVkuZunzlLgTD', '7aF09WaavZAmAWuUeYxlYD', '1DfFHyrenAJbqsLcpRiOD9', '42gJat74CEyy3jPliCVKOC', '53SnCRcGrB2UCqNGyYZhJD', '1CtQClLffhYcQKRIAtM0qm', '4iLqG9SeJSnt0cSPICSjxv', '38yBBH2jacvDxrznF7h08J', '3B54sVLJ402zGa6Xm4YGNe', '6jA8HL9i4QGzsj6fjoxp8Y', '3PEgB3fkiojxms35ntsTgs', '5aAx2yezTd8zXrkmtKl66Z', '2QffgsYVZkvMQsZmQdgf2L', '2HSydyyFoQCBRxj3sZcbpz', '06TuMCjDEYjTOJScCRzmjf']

In [6]:
audio_features_dict = {}
for song in songs:
        audio_features_dict[song] = spotify.audio_features("spotify:track:"+ str(song))[0]

In [11]:
audio_features_df = pd.DataFrame.from_dict(audio_features_dict, orient='index')

audio_features_df = audio_features_df.drop(columns=['type', 'id','uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
audio_features_df = audio_features_df.drop(columns=['key', 'mode', 'tempo'])

display(audio_features_df)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
15MTd64KUMG7CF6mOyovsQ,0.720,0.746,-5.567,0.0983,0.125000,0.000476,0.0929,0.511
3kGfazcbVvVkuZunzlLgTD,0.708,0.868,-3.287,0.0431,0.031300,0.000000,0.1220,0.720
7aF09WaavZAmAWuUeYxlYD,0.593,0.868,-3.573,0.2950,0.183000,0.000000,0.3650,0.496
1DfFHyrenAJbqsLcpRiOD9,0.597,0.567,-4.539,0.0342,0.561000,0.000005,0.1420,0.188
42gJat74CEyy3jPliCVKOC,0.768,0.858,-5.494,0.0482,0.191000,0.000032,0.6020,0.556
53SnCRcGrB2UCqNGyYZhJD,0.810,0.885,-4.583,0.1080,0.101000,0.000000,0.2680,0.447
1CtQClLffhYcQKRIAtM0qm,0.581,0.779,-6.178,0.0626,0.092500,0.000196,0.2300,0.172
4iLqG9SeJSnt0cSPICSjxv,0.774,0.626,-4.432,0.0432,0.096900,0.000031,0.0848,0.777
38yBBH2jacvDxrznF7h08J,0.734,0.418,-6.678,0.0425,0.012900,0.000000,0.0579,0.868
3B54sVLJ402zGa6Xm4YGNe,0.726,0.769,-5.043,0.1230,0.029300,0.010100,0.1040,0.733


In [47]:
# based on the audio features, create a similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(audio_features_df, audio_features_df)

# introduce some randomness so it skips some of the closest values, so that it doesn't pick the 
# absolute closest songs every time
# start with semnatic textual similarity 
# two vectors, compare how close they are to each other, could also just do an embedding 
# google colab gpu machines, or spin off an aws account, and spin off a gpu machine from aws or azure
# if you have access to the spotify api features, do it both ways
# once with word embeddings, once with music features, then combine them, depending on how much you want to weight them 

# create some metrics yourself for evaluation, or look at how many in playlists vs. outof plylist songs are recommended 
# vs. random sampling 

# signal processing for how to extract the classical features 

# 1: Get a sentence embedder
# 2: Put embeddings together in a latent space
# 3: Use KNN to get closest pairs
# 1: Get music features
# 2: Put music features in latent space
# 3: use KNN to get closest pairs
# 1: Combine two methods for song recommendations

# collaborative filtering

# create a series for the song titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(audio_features_df.index)

# function that takes in song title as input and returns the top 10 recommended songs
def recommendations(song, cosine_sim = cosine_sim):
        
        recommended_songs = []
        
        # gettin the index of the song that matches the title
        idx = indices[indices == song].index[0]
    
        # creating a Series with the similarity scores in descending order
        score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
        # getting the indexes of the 10 most similar songs
        top_10_indexes = list(score_series.iloc[1:11].index)
        
        # populating the list with the titles of the best 10 matching songs
        for i in top_10_indexes:
            recommended_songs.append(list(audio_features_df.index)[i])
            
        return recommended_songs

recommendations('15MTd64KUMG7CF6mOyovsQ')


# def wordMapfromSongVec(track_uri):
#     features = ['danceability','energy','speechiness','acousticness','liveness','valence']
#     songVec = spotify.audio_features("spotify:track:"+ str(track_uri))[0]
#     wordMap = {}
#     for feature in features:
#         wordMap[feature] = songVec[feature]
#     # turn audio features into text string where bigger values are more frequent
#     wordMap = {k: v * 100 for k, v in wordMap.items()}
    

['3PEgB3fkiojxms35ntsTgs',
 '2QffgsYVZkvMQsZmQdgf2L',
 '3B54sVLJ402zGa6Xm4YGNe',
 '5aAx2yezTd8zXrkmtKl66Z',
 '06TuMCjDEYjTOJScCRzmjf',
 '1CtQClLffhYcQKRIAtM0qm',
 '53SnCRcGrB2UCqNGyYZhJD',
 '38yBBH2jacvDxrznF7h08J',
 '4iLqG9SeJSnt0cSPICSjxv',
 '42gJat74CEyy3jPliCVKOC']